In [1]:
import src.lightning as l

import torch
import torch.utils.data as data
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint

In [2]:
L=10
N=10000
batch_size=100
checkpoint_path='./tensorboard/synth'

In [3]:
model = l.LightningSeq2Seq(L, L, warmup=200, ff_dim=256)

In [4]:
train = data.DataLoader(l.SynthDataset(L, N), batch_size=batch_size)
val = data.DataLoader(l.SynthDataset(L, N // 100), batch_size=batch_size)

In [5]:
trainer = pl.Trainer(
    default_root_dir=checkpoint_path,
    gpus=0,
    max_epochs=5,
    callbacks=[
        # Save the best checkpoint based on the maximum val_acc recorded. Saves only weights and not optimizer
        ModelCheckpoint(
            save_weights_only=True, mode="max", monitor="val_loss"
        ),
        # Log learning rate every epoch
        LearningRateMonitor("step")
    ],
    fast_dev_run=False
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [6]:
trainer.fit(model, train, val)

Missing logger folder: tensorboard/synth/lightning_logs

  | Name        | Type             | Params | In sizes         | Out sizes 
---------------------------------------------------------------------------------
0 | model       | TransformerModel | 22.1 M | [[1, 4], [1, 4]] | [1, 4, 10]
1 | loss_module | Seq2SeqLoss      | 0      | ?                | ?         
---------------------------------------------------------------------------------
22.1 M    Trainable params
0         Non-trainable params
22.1 M    Total params
88.457    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/home/lukasz/bin/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/lukasz/bin/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [7]:
%load_ext tensorboard

In [8]:
%tensorboard --logdir ./tensorboard/synth